In [ ]:
import json
import re

In [ ]:
RESULTS_DIR = "games_descriptions/base/output/test-moderator"

In [ ]:
with open(RESULTS_DIR + "/history16_44_54.json", "r") as f:
    history = json.load(f)

In [ ]:
def extract_deals(history: dict) -> list[dict[str, str]]:
    """
    Extract deals from the history of rounds.
    Gets the text between <DEAL> and </DEAL> in the public answer.
    Splits by commas to get the individual parts of the deal, then strips off the letters to just get the number values.
    """
    deals = []
    for round in history["rounds"]:
        public_answer = round["public_answer"]
        # Deal is between <DEAL> and </DEAL>
        deal_match = re.search(r"<DEAL>(.*?)</DEAL>", public_answer, re.DOTALL)
        if deal_match:
            deal = deal_match.group(1).strip()
            parts = deal.split(",")
            # Use regex to extract numbers from each part
            parts = [int(re.sub(r"[^\d.]", "", part.strip())) for part in parts if part.strip()]
            if parts:
                deals.append({"agent": round["agent"], "deal": parts})
            else:
                raise ValueError("Unable to parse deal")
        else:
            raise ValueError("No deal found")
    return deals

In [ ]:
deals = extract_deals(history)

In [ ]:
deals